In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report,balanced_accuracy_score
from collections import Counter
import itertools
import multiprocessing as mp
import random


In [1]:
!pip install catboost


[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: C:\Users\Shanttanu\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
# def build_dictionary(dictionary_file_location):
#     text_file = open(dictionary_file_location,"r")
#     full_dictionary = text_file.read().splitlines()
#     text_file.close()
#     return full_dictionary
# words = build_dictionary("words_250000_train.txt")


In [4]:
# dictionary = build_dictionary("words_250000_train.txt")


In [5]:
# sets = [set(x) for x in dictionary]
# #find longest word in dictionary
# max_len = 0
# w = ""
# for word in dictionary: 
#     if len(word) > max_len:
#         max_len = len(word)
#         w = word
# print(max_len)

29


In [3]:


# def get_combinations(letters):
#     combinations = []
#     for i in range(1,len(letters)+1):
#         combinations += list(itertools.combinations(letters, i))
#     return combinations
# nos = [len(x) for x in sets]
# freq = Counter(nos)
# summ = 0
# ss = "abcdefghijklmnopqrstuvwxyz"
# for i in freq:
#     summ += freq[i]*len(get_combinations(ss[:i]))
value = {"a":1,"b":2,"c":3,"d":4,"e":5,"f":6,"g":7,
         "h":8,"i":9,"j":10,"k":11,"l":12,"m":13,"n":14,
         "o":15,"p":16,"q":17,"r":18,"s":19,"t":20,"u":21,
         "v":22,"w":23,"x":24,"y":25,"z":26}

         
# def dataframemaker(dicc):    
#     dfs = []

#     for word in dicc:
#         for comb in get_combinations("".join(set(word))):
#             #create an array of -1s of length 80
#             num = np.full(80,-1)
#             guess = np.full(26,0)
#             #fill the array with the index of the letters in the word
#             k=len(word)
#             for i in range(k):
#                 if(word[i] in comb):
#                     num[i] = value[word[i]]
#                     num[80-k+i] = value[word[i]]
#                 else:
#                     num[i] = 0
#                     num[80-k+i] = 0
#                     guess[value[word[i]]-1] = 1
#             dfs.append(num.tolist()+guess.tolist())
    
#     return pd.DataFrame(dfs,dtype=np.int8)

In [4]:

# dataset = []

# for dicc in np.array_split(dictionary,10):
    
#     print("1 \n")

#     manager = mp.Manager()
#     pool = mp.Pool()

#     dictsplit = np.array_split(dicc,128)

#     datas = pool.map(dataframemaker,dictsplit)
#     pool.close()
#     pool.join()

#     finaldata = pd.concat(datas).reset_index(drop=True)

#     del datas

#     finaldata.columns = [str(x) for x in range(80)]+[x for x in "abcdefghijklmnopqrstuvwxyz"]
#     dataset.append(finaldata)
#     del finaldata


In [ ]:
# dataset = pd.concat(dataset).reset_index(drop=True)
# dataset.to_parquet("fd.parquet")

In [7]:
dataset = pd.read_parquet("fd.parquet")
alpha = "abcdefghijklmnopqrstuvwxyz"


In [6]:
!pip install pyarrow
!pip install fastparquet


[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: C:\Users\Shanttanu\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



                                              0.0/25.1 MB ? eta -:--:--
                                              0.4/25.1 MB 11.8 MB/s eta 0:00:03
     -                                        0.8/25.1 MB 10.2 MB/s eta 0:00:03
     -                                        1.0/25.1 MB 9.2 MB/s eta 0:00:03
     --                                       1.4/25.1 MB 9.0 MB/s eta 0:00:03
     ---                                      2.0/25.1 MB 9.1 MB/s eta 0:00:03
     ---                                      2.4/25.1 MB 9.0 MB/s eta 0:00:03
     ----                                     3.0/25.1 MB 10.1 MB/s eta 0:00:03
     -----                                    3.6/25.1 MB 10.0 MB/s eta 0:00:03
     ------                                   4.0/25.1 MB 9.8 MB/s eta 0:00:03
     ------                                   4.4/25.1 MB 9.6 MB/s eta 0:00:03
     -------                                  4.8/25.1 MB 9.5 MB/s eta 0:00:03
     --------                                 5.3/25.1


[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: C:\Users\Shanttanu\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
class MultiLabelCatBoostClassifier:
    def __init__(self, num_classes=26, catboost_params=None):
        # Initialize 26 CatBoost classifiers, one for each label
        self.classifiers = [CatBoostClassifier(iterations=1500) for _ in range(num_classes)]
        
    def fit(self, X, y):
        # X is the feature matrix, y is a binary matrix indicating label presence
        for i in range(len(self.classifiers)):
            # Train each classifier on the corresponding label
            # dd = y[y[alpha[i]] == 0].sample(len(y) - 2*len(y[y[alpha[i]] == 1])).index
            dd = []
            self.classifiers[i].fit(X.drop(dd), y[alpha[i]].drop(dd), verbose=100)
    
    def predict(self, X):
        # Predict probabilities for each label
        predictions = np.zeros((len(X), len(self.classifiers)))
        for i, clf in enumerate(self.classifiers):
            predictions[:, i] = clf.predict_proba(X)[:, 1]  # Probability of class '1'
        return predictions
    
    def save(self, filename):
        # Save the model to a pickle file
        with open(filename, 'wb') as file:
            pickle.dump(self, file)
    
    @classmethod
    def load(cls, filename):
        # Load the model from a pickle file
        with open(filename, 'rb') as file:
            model = pickle.load(file)
        return model

In [9]:
model = MultiLabelCatBoostClassifier()

#train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset[[str(x) for x in range(80)]], dataset[[x for x in "abcdefghijklmnopqrstuvwxyz"]], test_size=0.01, random_state=42)

model.fit(X_train,y_train)

MemoryError: Unable to allocate 2.44 GiB for an array with shape (26, 100725482) and data type int8

In [ ]:
#calculate accuracy between individual columns

def accuracy(y_pred, y_test):
    cols = y_test.columns
    x = []
    for i in range(len(cols)):
        print(cols[i]," : ",confusion_matrix(np.round(y_pred[:,i]),y_test[cols[i]]))
        print(cols[i]," : ",balanced_accuracy_score(np.round(y_pred[:,i]),y_test[cols[i]]))
        x.append(balanced_accuracy_score(np.round(y_pred[:,i]),y_test[cols[i]]))
    return np.mean(x)

accuracy(model.predict(X_test),y_test)

In [ ]:
model.save('models_trained/multilabel_catboost_model.pkl')

Testing 


In [ ]:
value_rev = {v: k for k, v in value.items()}
#array of size 80 with -1s
def prediction(tc):
    inp = np.array([-1 for x in range(80)])
    for i in range(len(tc)):
        inp[i] = value[tc[i]]
        inp[80-len(tc)+i] = value[tc[i]]

    inp = model.predict(np.array([inp]))[0]

    perm = ""
    prob = []
    for i in range(26):
        perm += alpha[np.argmax(inp)]
        prob.append(inp[np.argmax(inp)])
        inp[np.argmax(inp)] = -1
    return perm, prob

In [ ]:
#create a subset of 6 character x which contain only one '_' and the rest are alphabets and replace '_' with '*'
def create_substrings(trial,guessed=[],n=6,threshold=0.1,multiple=False):
    substring = []
    for i in range(len(trial)):
        if trial[i] == '_' and i >= 0 and i <= len(trial)-1 :
            if multiple:
                #replace all '_' with '.'
                substring.append((trial[max(i-n+1,0):i] + '.' + trial[i+1:min(i+n,len(trial))]).replace("_","*"))
            
            if not multiple and (i==0 or trial[i-1] != '_') and (i==len(trial)-1 or trial[i+1] != '_'): 
                substring.append(trial[max(i-n+1,0):i] + '.' + trial[i+1:min(i+n,len(trial))])
    
    #if any string in substring has more than two '_' then remove it
    substrings = []
    for trial in substring:
        for i in range(0,len(trial)-n+1):
            substrings.append(trial[i:i+n])
    
    if multiple:
        substrings  = [x for x in substrings if x.count('*') < 2]
    else:
        substrings  = [x for x in substrings if x.count('_') == 0]
    
    substrings  = [x for x in substrings if len(x) == n]

    if(substrings == []):
        # print("No substring found")
        return None,None
    
    ans = []
    
    for x in substrings:
        ind = x.index('.')
        letters = []

        if multiple:
            x = x.replace("*",".")
        
        for word in words:
            if re.search(x,word):
                l = word[re.search(x,word).start()+ind]
                if l not in guessed:
                    letters.append(l)

        if letters == []:
            continue
        
        b = [[a[0],a[1]/len(letters),a[1]] for a in Counter(letters).most_common()]
        ans.append(b)
    
    if ans == []:
        # print("No substring found")
        return None,None
    
    #create an array with 26 0s representing all alphabets and then add each letter's probability to it
    # print(ans)
    bb = np.array([0.0 for aaaaa in range(26)])

    for x in ans:
        for y in x:
            bb[value[y[0]]-1] += y[1]

    for i in range(len(bb)):
        bb[i] = bb[i]/bb.sum()
    # print(bb)
    if bb[np.argmax(bb)]<threshold:
        return None,None
    #return the most probable letter
    return value_rev[np.argmax(bb)+1],bb[np.argmax(bb)]

trial ="sur_ers"
guess = ['d','a','r','l','t','m','p','g']

print(create_substrings(trial,guess,8,multiple=True))
# print("not8")
print(create_substrings(trial,guess,7,multiple=True))
# print("not7")
print(create_substrings(trial,guess,6,multiple=True))
# print("not6")
print(create_substrings(trial,guess,5,multiple=True))
# print("not5")
print(create_substrings(trial,guess,4,multiple=True))
# print("not4")
print(create_substrings(trial,guess,3))

In [ ]:
def guessing(word, guessed_letters,verbose = False):

    aa,probs = prediction(word)
    # print(aa,probs)

    pred_prob = 0

    for i in range(len(aa)):
        if(aa[i] not in guessed_letters):
            pred_letter = aa[i]
            pred_prob = probs[i]
            break
    
    a,prob = create_substrings(word,guessed_letters,8,multiple=True)
    if a != None and prob > pred_prob:
        if verbose:
            print("used8",a)
        return a
    
    [a,prob] = create_substrings(word,guessed_letters,7,multiple=True)
    if a != None and prob > pred_prob:
        if verbose:
            print("used7",a)
        return a
    
    [a,prob] = create_substrings(word,guessed_letters,6,multiple=True)
    if a != None and prob > pred_prob:
        if verbose:
            print("used6",a)
        return a

    [a,prob] = create_substrings(word,guessed_letters,5,multiple=True)
    if a != None and prob > pred_prob:
        if verbose:
            print("used5",a)
        return a

    [a,prob] = create_substrings(word,guessed_letters,4,multiple=True)
    if a != None and prob > pred_prob:
        if verbose:
            print("used4",a)
        return a
    
    [a,prob] = create_substrings(word,guessed_letters,3)
    if a != None and prob > pred_prob:
        if verbose:
            print("used3",a)
        return a
    
    return pred_letter

In [ ]:
#create a string contianing the permutation of "abcdefghijklmnopqrstuvwxyz" in descending order of values in the np array of length 26

def tries_taken(word,verbose=False,limit=6):
    tc = ['_' for x in word]
    tc = "".join(tc)
    guesslist = []
    count=0
    # print(tc)
    while(1):
        if(count>=limit):
            return count
        i = guessing(tc, guesslist,verbose=verbose)
        guesslist.append(i)
        if(i in word):
            #replace all occurences of i in tc with i at positions in word
            if(verbose):
                print("correct guess : ",i)
            for j in range(len(word)):
                if(word[j]==i):
                    tc = list(tc)
                    tc[j]=i
                    tc = "".join(tc)
            if(tc == word):
                print("word guessed completely : ",tc)
                return count
        else:
            # print("wrong guess : ",i)
            count+=1

        if(verbose):
            print(tc , i)

In [ ]:
random_words = random.sample(words,2)
aa = []
for i in random_words:
    # print(i)
    aa.append(tries_taken(i,verbose=True,limit=6))
#percentage of values less than 6 in aa

count = 0
for i in aa:
    if(i<7):
        count+=1
print(count/10)